In [50]:
from typing import Callable, List
import numpy as np
from numpy import random
import json
import subprocess
import sys
import pickle
from pathlib import Path
from os import path, _exit
from tensorflow import keras
from tensorflow.keras import layers
import onnx
import keras2onnx
from tensorflow.python.keras.engine.sequential import Sequential
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from es import OpenES
from shutil import rmtree
import time
from gomokutrainer import Evaluator, create_model, get_param_count

In [51]:
input_shape = (15,15,2)
input_len = input_shape[0] * input_shape[1] * input_shape[2]
def create_custom_model():
    return keras.Sequential(
        [
            keras.Input(shape=input_len, name="input"),
            layers.Reshape(input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), kernel_initializer=keras.initializers.HeNormal()),
            layers.LeakyReLU(alpha=0.2),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(32, kernel_size=(3, 3), kernel_initializer=keras.initializers.HeNormal()),
            layers.LeakyReLU(alpha=0.2),
            layers.Flatten(),
            layers.Dense(32, kernel_initializer=keras.initializers.HeNormal()),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(1, name="output")
        ]
    )
create_model_function = create_custom_model

In [52]:
NPARAMS = sum([np.array(layer).size for layer in create_model().get_weights()])
solver = OpenES(get_param_count(),              # number of model parameters
                sigma_init=0.1,            # initial standard deviation
                sigma_decay=0.9995,         # don't anneal standard deviation
                sigma_limit=0.01,
                learning_rate=0.01,        # learning rate for standard deviation
                learning_rate_decay = 0.999, # annealing the learning rate
                learning_rate_limit = 0.0005,
                popsize=1,       # population size
                antithetic=False,          # whether to use antithetic sampling
                weight_decay=0.00,         # weight decay coefficient
                rank_fitness=True,         # use rank rather than fitness numbers
                forget_best=True)

In [53]:
list_of_flat_weights = solver.ask()
evaluator = Evaluator(create_model_function, "")
weights = evaluator.wrap_weights(list_of_flat_weights[0])

In [54]:
model = create_model_function()


In [55]:
model.set_weights(weights)

In [56]:
input_shape = model.layers[0].input_shape
print(input_shape)
data = random.rand(10000,input_shape[1]).astype(np.single)
print(data)

(None, 450)
[[0.83389354 0.6160088  0.6465018  ... 0.73201764 0.32026765 0.10031243]
 [0.0525916  0.04148839 0.12899868 ... 0.27170902 0.0797263  0.02913417]
 [0.37547097 0.34349748 0.8275108  ... 0.08299976 0.47801086 0.55089444]
 ...
 [0.382114   0.69373524 0.12343797 ... 0.4196698  0.17195965 0.53858334]
 [0.70352787 0.84197146 0.07894789 ... 0.13899617 0.5148081  0.5298883 ]
 [0.81268966 0.47035292 0.73294693 ... 0.73311883 0.22178224 0.04397286]]


In [57]:
predictions = model.predict(data)
predictions_sigmoid = keras.activations.sigmoid(predictions).numpy()
#print(predicitons)
#print()
print("max value: ",max(predictions))
print("max sigmoid value: ", max(predictions_sigmoid))
print("min value: ",min(predictions))
print("min sigmoid value: ", min(predictions_sigmoid))

max value:  [0.6145785]
max sigmoid value:  [0.6489845]
min value:  [-0.02984944]
min sigmoid value:  [0.49253818]
